# Vector stores and retrievers - 向量存储和检索器

本教程将帮助您熟悉 LangChain 的向量存储和检索器抽象。这些抽象旨在支持从（向量）数据库和其他来源检索数据，以便与 LLM 工作流集成。它们对于获取数据以作为模型推理的一部分进行推理的应用程序非常重要，例如检索增强生成 (RAG) 的情况。

## 概念

本指南重点介绍文本数据的检索。我们将介绍以下概念：

 - 文档
 - 向量存储
 - 检索器

# 文档

LangChain 实现了 Document 抽象，旨在表示文本单元和相关元数据。它有两个属性：

 - page_content：表示内容的字符串
 - metadata：包含任意元数据的字典
   
metadata 属性可以捕获有关文档来源、与其他文档的关系以及其他信息的信息。请注意，单个 Document 对象通常代表较大文档的一部分。

让我们生成一些示例文档：

In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="狗是很好的伙伴，以忠诚和友好而闻名。",
        metadata={"source": "哺乳动物-宠物-doc"},
    ),
    Document(
        page_content="猫是独立的宠物，它们通常喜欢有自己的空间。",
        metadata={"source": "哺乳动物-宠物-doc"},
    ),
    Document(
        page_content="金鱼是初学者喜欢的宠物，需要相对简单的照顾。",
        metadata={"source": "鱼-宠物-doc"},
    ),
    Document(
        page_content="鹦鹉是一种聪明的鸟类，能够模仿人类的语言。",
        metadata={"source": "鱼-宠物-doc"},
    ),
    Document(
        page_content="兔子是群居动物，需要足够的空间来跳跃。",
        metadata={"source": "哺乳动物-宠物-doc"},
    ),
]

这里我们生成了五个文档，包含指示三个不同“来源”的元数据。

## Vector stores - 向量存储

向量搜索是存储和搜索非结构化数据（例如非结构化文本）的常用方法。其理念是存储与文本相关的数字向量。给定一个查询，我们可以将其嵌入为相同维度的向量，并使用向量相似性指标来识别存储中的相关数据。  

LangChain VectorStore 对象包含用于将文本和 Document 对象添加到存储区以及使用各种相似度指标查询它们的方法。它们通常使用嵌入模型进行初始化，这些模型决定了如何将文本数据转换为数字向量。  

LangChain 包含一套与不同向量存储技术的集成。一些向量存储由提供商（例如，各种云提供商）托管，需要特定凭据才能使用；一些（例如 Postgres）在单独的基础架构中运行，可以在本地或通过第三方运行；其他可以在内存中运行以处理轻量级工作负载。在这里，我们将演示使用 Chroma 的 LangChain VectorStores 的使用，其中包括内存实现。 

要实例化向量存储，我们通常需要提供一个嵌入模型来指定如何将文本转换为数字向量。在这里我们将使用 百川的词嵌入模型。  

In [2]:
# 获取你的智谱 API Key
# 在当前文件下创建一个.env文件，将api-key复制进去，如ZHIPUAI_API_KEY = "api-key"

from dotenv import load_dotenv,find_dotenv
import os 
_ = load_dotenv(find_dotenv())

In [3]:
# from langchain_openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(
#     base_url="https://open.bigmodel.cn/api/paas/v4",
#     api_key=os.environ["ZHIPUAI_API_KEY"],
#     model="Embedding-2"
# )

In [4]:
from langchain_community.embeddings import BaichuanTextEmbeddings

embeddings = BaichuanTextEmbeddings(baichuan_api_key=os.environ["BAICHUAN_API_KEY"])

In [5]:
text = "你好"
query_result = embeddings.embed_query(text)
query_result[:5]

[-0.008814517, 0.06622468, -0.074492484, 0.04030321, -0.0010948664]

### 安装向量存储库 Chroma

In [6]:
# pip install langchain-chroma

In [7]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
)

在此处调用 .from_documents 会将文档添加到向量存储中。VectorStore 实现了添加文档的方法，这些方法也可以在对象实例化后调用。大多数实现都允许您连接到现有的向量存储
例如，通过提供客户端、索引名称或其他信息。有关更多详细信息，请参阅特定集成的文档。

一旦我们实例化了包含文档的 VectorStore，我们就可以对其进行查询。VectorStore 包含以下查询方法：

 - 同步和异步
 - 通过字符串或向量查询
 - 返回和不返回相似度分数
 - 通过相似度和最大边际相关性（以平衡相似度和查询与检索结果的多样性）

这些方法通常会在其输出中包含一个 Document 对象列表。

### 示例

根据与字符串查询的相似性返回文档：

In [8]:
vectorstore.similarity_search("小狗")

[Document(page_content='狗是很好的伙伴，以忠诚和友好而闻名。', metadata={'source': '哺乳动物-宠物-doc'}),
 Document(page_content='金鱼是初学者喜欢的宠物，需要相对简单的照顾。', metadata={'source': '鱼-宠物-doc'}),
 Document(page_content='鹦鹉是一种聪明的鸟类，能够模仿人类的语言。', metadata={'source': '鱼-宠物-doc'}),
 Document(page_content='猫是独立的宠物，它们通常喜欢有自己的空间。', metadata={'source': '哺乳动物-宠物-doc'})]

### 异步查询 - Async query

In [9]:
await vectorstore.asimilarity_search("鹦鹉")

[Document(page_content='鹦鹉是一种聪明的鸟类，能够模仿人类的语言。', metadata={'source': '鱼-宠物-doc'}),
 Document(page_content='金鱼是初学者喜欢的宠物，需要相对简单的照顾。', metadata={'source': '鱼-宠物-doc'}),
 Document(page_content='兔子是群居动物，需要足够的空间来跳跃。', metadata={'source': '哺乳动物-宠物-doc'}),
 Document(page_content='猫是独立的宠物，它们通常喜欢有自己的空间。', metadata={'source': '哺乳动物-宠物-doc'})]

### 返回分数 - Return scores

In [10]:
vectorstore.similarity_search_with_score("鹦鹉")

[(Document(page_content='鹦鹉是一种聪明的鸟类，能够模仿人类的语言。', metadata={'source': '鱼-宠物-doc'}),
  0.7093302607536316),
 (Document(page_content='金鱼是初学者喜欢的宠物，需要相对简单的照顾。', metadata={'source': '鱼-宠物-doc'}),
  1.1786209344863892),
 (Document(page_content='兔子是群居动物，需要足够的空间来跳跃。', metadata={'source': '哺乳动物-宠物-doc'}),
  1.2764108180999756),
 (Document(page_content='猫是独立的宠物，它们通常喜欢有自己的空间。', metadata={'source': '哺乳动物-宠物-doc'}),
  1.3013885021209717)]

### 根据与嵌入式查询的相似性返回文档

In [11]:
embedding = embeddings.embed_query("猫")

vectorstore.similarity_search_by_vector(embedding)

[Document(page_content='猫是独立的宠物，它们通常喜欢有自己的空间。', metadata={'source': '哺乳动物-宠物-doc'}),
 Document(page_content='金鱼是初学者喜欢的宠物，需要相对简单的照顾。', metadata={'source': '鱼-宠物-doc'}),
 Document(page_content='鹦鹉是一种聪明的鸟类，能够模仿人类的语言。', metadata={'source': '鱼-宠物-doc'}),
 Document(page_content='狗是很好的伙伴，以忠诚和友好而闻名。', metadata={'source': '哺乳动物-宠物-doc'})]

## Retrievers - 检索器

LangChain VectorStore 对象不继承 Runnable，因此无法立即集成到 LangChain 表达式语言链中。

LangChain Retrievers 是 Runnable，因此它们实现了一组标准方法（例如，同步和异步调用和批处理操作），并设计为集成到 LCEL 链中。

我们可以自己创建一个简单的版本，而无需继承 Retriever。如果我们选择要使用什么方法来检索文档，我们可以轻松创建一个可运行的版本。  

下面我们将围绕 similarity_search 方法构建一个：

In [12]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["猫", "鲨鱼"])

[[Document(page_content='猫是独立的宠物，它们通常喜欢有自己的空间。', metadata={'source': '哺乳动物-宠物-doc'})],
 [Document(page_content='金鱼是初学者喜欢的宠物，需要相对简单的照顾。', metadata={'source': '鱼-宠物-doc'})]]

vectorstore 实现了一个 as_retriever 方法，该方法将生成一个 Retriever，特别是 VectorStoreRetriever。  
这些检索器包括特定的 search_type 和 search_kwargs 属性，用于标识要调用底层向量存储的哪些方法以及如何参数化它们。  
例如，我们可以使用以下命令复制上述内容：

In [13]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["狗", "鸟"])

[[Document(page_content='狗是很好的伙伴，以忠诚和友好而闻名。', metadata={'source': '哺乳动物-宠物-doc'})],
 [Document(page_content='鹦鹉是一种聪明的鸟类，能够模仿人类的语言。', metadata={'source': '鱼-宠物-doc'})]]

VectorStoreRetriever 支持“相似性”（默认）、“mmr”（最大边际相关性，如上所述）和“similarity_score_threshold”搜索类型。  
我们可以使用后者根据相似性分数对检索器输出的文档进行阈值处理。

检索器可以轻松融入更复杂的应用程序，例如检索增强生成 (RAG) 应用程序，该应用程序将给定的问题与检索到的上下文组合成 LLM 的提示。  
下面我们展示一个最小示例。

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://api.baichuan-ai.com/v1",
    api_key=os.environ["BAICHUAN_API_KEY"],
    model="Baichuan4",
)

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
仅使用提供的上下文来回答这个问题。

{question}

上下文语境:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [16]:
response = rag_chain.invoke("告诉我关于猫的事")

print(response.content)

根据提供的上下文，猫是一种独立的宠物，它们通常喜欢拥有自己的空间。


In [17]:
resp = rag_chain.invoke("告诉我关于鹦鹉的事")
resp.content

'根据提供的上下文，鹦鹉是一种非常聪明的鸟类，它们有能力模仿人类的语言。这种能力使得鹦鹉成为人们喜爱的宠物之一。'

In [18]:
resp = rag_chain.invoke("告诉我关于小兔子的事")
resp.content

'根据提供的上下文，小兔子是一种群居动物，这意味着它们喜欢与其他兔子一起生活。此外，小兔子需要足够的空间来进行跳跃活动，这是它们的一种自然行为。这些信息可以帮助你了解如何照顾和饲养小兔子作为宠物。'